In [ ]:
pip install gspread

In [ ]:
pip install oauth2client

In [ ]:
pip install gspread_dataframe

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('testtask-315809-dc167efb230c.json', scope)
client = gspread.authorize(creds)

sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ/edit#gid=1167027454')



In [2]:
up_sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1W2xEH74QNMb7W3IdgwUtLD9_miiNfQAiE6tJvcoOCaI/edit#gid=0')

In [3]:
from gspread_dataframe import set_with_dataframe

In [4]:
import pandas as pd

In [5]:
transactions = pd.DataFrame.from_dict(sheet.worksheet('transactions').get_all_records())
clients = pd.DataFrame.from_dict(sheet.worksheet('clients').get_all_records())
managers = pd.DataFrame.from_dict(sheet.worksheet('managers').get_all_records())
leads = pd.DataFrame.from_dict(sheet.worksheet('leads').get_all_records())

In [88]:
final = leads.merge(managers, left_on='l_manager_id', right_on='manager_id')
final.loc[final.l_client_id != '00000000-0000-0000-0000-000000000000', 'num_of_garbage'] = 1
final = final[['lead_id', 'created_at', 'd_manager', 'd_club', 'd_utm_source', 'num_of_garbage']]

In [89]:
leads_mrg = leads.merge(clients, left_on='l_client_id', right_on='client_id', how='left')
leads_mrg = leads_mrg.merge(transactions, on='l_client_id', how='left')[['lead_id', 'created_at_x', 'created_at_y', 'created_at', 'd_utm_source', 'l_manager_id_x']]
leads_mrg = leads_mrg.groupby(by=['lead_id'], as_index=False).min()
leads_mrg.fillna('9999-12-31 00:00:00', inplace=True)
leads_mrg = leads_mrg[(leads_mrg.created_at_x <= leads_mrg.created_at_y) & (leads_mrg.created_at_x <= leads_mrg.created_at)]
leads_mrg = leads_mrg.merge(managers, left_on='l_manager_id_x', right_on='manager_id')
leads_mrg = leads_mrg[['lead_id']]
leads_mrg['num_of_new'] = 1

In [90]:
final = final.merge(leads_mrg, on='lead_id', how='left')

In [91]:
leads_trans = leads.merge(transactions, on='l_client_id', how='inner')
leads_trans['created_at_x'] = pd.to_datetime(leads_trans['created_at_x'])
leads_trans['created_at_y'] = pd.to_datetime(leads_trans['created_at_y'])
leads_trans = leads_trans[(leads_trans.created_at_x + pd.Timedelta(days=7) >= leads_trans.created_at_y) & (leads_trans.created_at_x <= leads_trans.created_at_y)]
leads_trans = leads_trans.groupby(by=['lead_id'], as_index=False).min()
leads_trans = leads_trans.merge(managers, left_on='l_manager_id', right_on='manager_id')
leads_trans = leads_trans[['lead_id']]
leads_trans['num_of_customers'] = 1

In [92]:
final = final.merge(leads_trans, on='lead_id', how='left')

In [93]:
leads_trans = leads.merge(transactions, on='l_client_id', how='inner')
leads_trans = leads_trans.groupby(by=['l_client_id']).agg({'created_at_x': 'max', 'created_at_y': 'min'}).reset_index()
leads_trans = leads_trans[leads_trans.created_at_x >= leads_trans.created_at_y][['l_client_id', 'created_at_x']]
leads_trans.rename(columns={'created_at_x': 'created_at'}, inplace=True)
leads_trans = leads.merge(leads_trans, on=['l_client_id', 'created_at']).merge(transactions, on='l_client_id', how='inner')
leads_trans['created_at_x'] = pd.to_datetime(leads_trans['created_at_x'])
leads_trans['created_at_y'] = pd.to_datetime(leads_trans['created_at_y'])
leads_trans = leads_trans[(leads_trans.created_at_x + pd.Timedelta(days=7) >= leads_trans.created_at_y) & (leads_trans.created_at_x <= leads_trans.created_at_y)]
leads_trans = leads_trans.merge(managers, left_on='l_manager_id', right_on='manager_id')
leads_trans = leads_trans[['lead_id', 'm_real_amount']]

In [94]:
final = final.merge(leads_trans, on='lead_id', how='left')

In [95]:
final.fillna(0, inplace=True)
final.loc[final.d_utm_source == '', 'd_utm_source'] = 'no source'
final.num_of_garbage = final.num_of_garbage.astype(int)
final.num_of_new = final.num_of_new.astype(int)
final.num_of_customers = final.num_of_customers.astype(int)
final.m_real_amount = final.m_real_amount.astype(int)

In [97]:
set_with_dataframe(up_sheet.sheet1, final)